In [32]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df

import datetime
from datetime import datetime as dt
import matplotlib.pyplot as plt

import os


#### read NARR Final-Data-Filter

In [120]:
print(os.getcwd())
DATA_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Data/'
RESULTS_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Results/'
CODE_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code/'

NARR_DATA_FOLDER = DATA_FOLDER + 'NARR/'
NARR_RAW_DATA_FOLDER = NARR_DATA_FOLDER + 'Raw-Data/'
NARR_FINAL_DATA_FOLDER = NARR_DATA_FOLDER + 'Final-Data/'
NARR_FINAL_DATA_DONE_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Done/'

NARR_FINAL_DATA_FILTER_FOLDER = NARR_DATA_FOLDER + 'Final-Data-Filter/'


/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code


In [127]:
narr_final_dirs = os.listdir(NARR_FINAL_DATA_FILTER_FOLDER)
print(narr_final_dirs, len(narr_final_dirs))
narr_final_dirs.remove('.DS_Store')
narr_final_dirs = [name for name in narr_final_dirs if '.xlsx' not in name]
print(narr_final_dirs, len(narr_final_dirs))

var_names = np.sort(narr_final_dirs)
print(var_names, len(var_names))

['uwnd', '.DS_Store', 'albedo', 'pottmp', 'air', 'dpt', 'tcdc', 'rhum', 'apcp', 'vwnd', 'acpcp'] 11
['uwnd', 'albedo', 'pottmp', 'air', 'dpt', 'tcdc', 'rhum', 'apcp', 'vwnd', 'acpcp'] 10
['acpcp' 'air' 'albedo' 'apcp' 'dpt' 'pottmp' 'rhum' 'tcdc' 'uwnd' 'vwnd'] 10


In [122]:
AIR_TEMP = 'air' # 1.Daily surface air temp. (K)
PRECIP_AMT = 'apcp' # 2.Daily accumulated total surface precip. (kg/m2)
CONV_PRECIP_AMT = 'acpcp' # 3.Daily accumulated conv. surface precip. (kg/m2)
TOTAL_CLOUD_COVER = 'tcdc' # 4.Daily forecast total cloud cover. (%)
UWND = 'uwnd' # 5.Daily u-wind eastward. (m/s)
DEW_POINT_TEMP = 'dpt' # 6.Daily dew point temp. (K)
ALBEDO = 'albedo' # 7.Daily surface albedo. (%)
POT_TEMP = 'pottmp' # 8.Daily potential surface temp. (K)
VWND = 'vwnd' # 9.Daily v-wind nothward. (m/s)
RH = 'rhum' # 10.Daily RH. (%)

VAR_NUMBER = 0


In [123]:
narr_final_files = os.listdir(NARR_FINAL_DATA_FILTER_FOLDER + var_names[VAR_NUMBER])
narr_final_files.remove('.DS_Store')
narr_final_files = np.sort(narr_final_files)
print(narr_final_files, len(narr_final_files))

['acpcp.2011.1.1.xlsx' 'acpcp.2011.1.10.xlsx' 'acpcp.2011.1.11.xlsx' ...
 'acpcp.2022.9.7.xlsx' 'acpcp.2022.9.8.xlsx' 'acpcp.2022.9.9.xlsx'] 4383


#### read all files of a variable -> store in a df [plant_code, day, month, year, value]
#### do the above for all 10 variables -> so, 10 dfs (1 for each var)

In [124]:
FROM_NARR_FINAL_DATA_FILE_NUMBER = 0 # starting from 0-(NUMBER_NARR_RAW-1)
TO_NARR_FINAL_DATA_FILE_NUMBER = 2557 # starting from 1-2557


In [147]:
PLANT_CODE = 'plant_code'
LATITUDE = 'lat'
LONGITUDE = 'lon'
MONTH = 'month'
YEAR = 'year'
DAY = 'day'

years_list = np.arange(2011, 2023)
years_list

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022])

In [148]:
REQUIRED_YEAR = 2015
REQUIRED_MONTH_LIST = np.arange(1, 13)
REQUIRED_YEARS_LIST = [2011, 2012, 2013, 2014, 2015]
REQUIRED_MONTH_LIST, REQUIRED_YEAR, REQUIRED_YEARS_LIST

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 2015,
 [2011, 2012, 2013, 2014, 2015])

In [149]:
ANOMALOUS_VAR_LIST = ['air', 'uwnd', 'vwnd', 'rhum', 'pottmp', 'dpt']
NORMAL_VAR_LIST = ['acpcp', 'apcp', 'tcdc', 'albedo']

In [151]:
# var_yrly_df_dict = {}

for var_ in var_names[-3:]:
    print(var_, '##############')
    narr_final_files = os.listdir(NARR_FINAL_DATA_FILTER_FOLDER + var_)
    narr_final_files.remove('.DS_Store')
    narr_final_files = np.sort(narr_final_files)
    print(len(narr_final_files), narr_final_files[0])
    for REQUIRED_YEAR in REQUIRED_YEARS_LIST:
        for mon_ in REQUIRED_MONTH_LIST:
            required_narr_final_files = [file for file in narr_final_files if (
                '.' + str(REQUIRED_YEAR) + '.' + str(mon_) + '.' in file)]
            print(len(required_narr_final_files))#, required_narr_final_files[0])
            var_df = df({PLANT_CODE: [], LATITUDE: [], LONGITUDE: [],
                         YEAR: [], MONTH: [], DAY: [], var_: []})
            for file in required_narr_final_files:
                print(file)
                if var_ in ANOMALOUS_VAR_LIST:
                    _, _, year, month, day, _ = file.split('.')
                else:
                    _, year, month, day, _ = file.split('.')
                narr_daily_var_df = pd.read_excel(
                    NARR_FINAL_DATA_FILTER_FOLDER + var_ + '/' + file)
                for ind, row in narr_daily_var_df.iterrows():
                    append_dict = {PLANT_CODE: [row[PLANT_CODE]],
                                   LATITUDE: [row[LATITUDE]],
                                   LONGITUDE: [row[LONGITUDE]],
                                   YEAR: [int(year)],
                                   MONTH: [int(month)],
                                   DAY: [int(day)],
                                   var_: [row[var_]]}
                    var_df = pd.concat([var_df, df(append_dict)])
            var_df.reset_index(inplace=True, drop=True)
            print(var_df.shape)
            filename = var_ + '.' + str(REQUIRED_YEAR) + '.' + str(mon_)
            var_df.to_excel(
                NARR_FINAL_DATA_FILTER_FOLDER + filename +'.xlsx',
                index=False)    

    
    



tcdc ##############
4382 tcdc.2011.1.1.xlsx
31
tcdc.2011.1.1.xlsx
tcdc.2011.1.10.xlsx
tcdc.2011.1.11.xlsx
tcdc.2011.1.12.xlsx
tcdc.2011.1.13.xlsx
tcdc.2011.1.14.xlsx
tcdc.2011.1.15.xlsx
tcdc.2011.1.16.xlsx
tcdc.2011.1.17.xlsx
tcdc.2011.1.18.xlsx
tcdc.2011.1.19.xlsx
tcdc.2011.1.2.xlsx
tcdc.2011.1.20.xlsx
tcdc.2011.1.21.xlsx
tcdc.2011.1.22.xlsx
tcdc.2011.1.23.xlsx
tcdc.2011.1.24.xlsx
tcdc.2011.1.25.xlsx
tcdc.2011.1.26.xlsx
tcdc.2011.1.27.xlsx
tcdc.2011.1.28.xlsx
tcdc.2011.1.29.xlsx
tcdc.2011.1.3.xlsx
tcdc.2011.1.30.xlsx
tcdc.2011.1.31.xlsx
tcdc.2011.1.4.xlsx
tcdc.2011.1.5.xlsx
tcdc.2011.1.6.xlsx
tcdc.2011.1.7.xlsx
tcdc.2011.1.8.xlsx
tcdc.2011.1.9.xlsx
(65999, 7)
28
tcdc.2011.2.1.xlsx
tcdc.2011.2.10.xlsx
tcdc.2011.2.11.xlsx
tcdc.2011.2.12.xlsx
tcdc.2011.2.13.xlsx
tcdc.2011.2.14.xlsx
tcdc.2011.2.15.xlsx
tcdc.2011.2.16.xlsx
tcdc.2011.2.17.xlsx
tcdc.2011.2.18.xlsx
tcdc.2011.2.19.xlsx
tcdc.2011.2.2.xlsx
tcdc.2011.2.20.xlsx
tcdc.2011.2.21.xlsx
tcdc.2011.2.22.xlsx
tcdc.2011.2.23.xlsx
tcdc.2011.

tcdc.2012.2.13.xlsx
tcdc.2012.2.14.xlsx
tcdc.2012.2.15.xlsx
tcdc.2012.2.16.xlsx
tcdc.2012.2.17.xlsx
tcdc.2012.2.18.xlsx
tcdc.2012.2.19.xlsx
tcdc.2012.2.2.xlsx
tcdc.2012.2.20.xlsx
tcdc.2012.2.21.xlsx
tcdc.2012.2.22.xlsx
tcdc.2012.2.23.xlsx
tcdc.2012.2.24.xlsx
tcdc.2012.2.25.xlsx
tcdc.2012.2.26.xlsx
tcdc.2012.2.27.xlsx
tcdc.2012.2.28.xlsx
tcdc.2012.2.29.xlsx
tcdc.2012.2.3.xlsx
tcdc.2012.2.4.xlsx
tcdc.2012.2.5.xlsx
tcdc.2012.2.6.xlsx
tcdc.2012.2.7.xlsx
tcdc.2012.2.8.xlsx
tcdc.2012.2.9.xlsx
(61741, 7)
31
tcdc.2012.3.1.xlsx
tcdc.2012.3.10.xlsx
tcdc.2012.3.11.xlsx
tcdc.2012.3.12.xlsx
tcdc.2012.3.13.xlsx
tcdc.2012.3.14.xlsx
tcdc.2012.3.15.xlsx
tcdc.2012.3.16.xlsx
tcdc.2012.3.17.xlsx
tcdc.2012.3.18.xlsx
tcdc.2012.3.19.xlsx
tcdc.2012.3.2.xlsx
tcdc.2012.3.20.xlsx
tcdc.2012.3.21.xlsx
tcdc.2012.3.22.xlsx
tcdc.2012.3.23.xlsx
tcdc.2012.3.24.xlsx
tcdc.2012.3.25.xlsx
tcdc.2012.3.26.xlsx
tcdc.2012.3.27.xlsx
tcdc.2012.3.28.xlsx
tcdc.2012.3.29.xlsx
tcdc.2012.3.3.xlsx
tcdc.2012.3.30.xlsx
tcdc.2012.3.31.xl

tcdc.2013.3.20.xlsx
tcdc.2013.3.21.xlsx
tcdc.2013.3.22.xlsx
tcdc.2013.3.23.xlsx
tcdc.2013.3.24.xlsx
tcdc.2013.3.25.xlsx
tcdc.2013.3.26.xlsx
tcdc.2013.3.27.xlsx
tcdc.2013.3.28.xlsx
tcdc.2013.3.29.xlsx
tcdc.2013.3.3.xlsx
tcdc.2013.3.30.xlsx
tcdc.2013.3.31.xlsx
tcdc.2013.3.4.xlsx
tcdc.2013.3.5.xlsx
tcdc.2013.3.6.xlsx
tcdc.2013.3.7.xlsx
tcdc.2013.3.8.xlsx
tcdc.2013.3.9.xlsx
(65999, 7)
30
tcdc.2013.4.1.xlsx
tcdc.2013.4.10.xlsx
tcdc.2013.4.11.xlsx
tcdc.2013.4.12.xlsx
tcdc.2013.4.13.xlsx
tcdc.2013.4.14.xlsx
tcdc.2013.4.15.xlsx
tcdc.2013.4.16.xlsx
tcdc.2013.4.17.xlsx
tcdc.2013.4.18.xlsx
tcdc.2013.4.19.xlsx
tcdc.2013.4.2.xlsx
tcdc.2013.4.20.xlsx
tcdc.2013.4.21.xlsx
tcdc.2013.4.22.xlsx
tcdc.2013.4.23.xlsx
tcdc.2013.4.24.xlsx
tcdc.2013.4.25.xlsx
tcdc.2013.4.26.xlsx
tcdc.2013.4.27.xlsx
tcdc.2013.4.28.xlsx
tcdc.2013.4.29.xlsx
tcdc.2013.4.3.xlsx
tcdc.2013.4.30.xlsx
tcdc.2013.4.4.xlsx
tcdc.2013.4.5.xlsx
tcdc.2013.4.6.xlsx
tcdc.2013.4.7.xlsx
tcdc.2013.4.8.xlsx
tcdc.2013.4.9.xlsx
(63870, 7)
31
tcdc.201

tcdc.2014.4.26.xlsx
tcdc.2014.4.27.xlsx
tcdc.2014.4.28.xlsx
tcdc.2014.4.29.xlsx
tcdc.2014.4.3.xlsx
tcdc.2014.4.30.xlsx
tcdc.2014.4.4.xlsx
tcdc.2014.4.5.xlsx
tcdc.2014.4.6.xlsx
tcdc.2014.4.7.xlsx
tcdc.2014.4.8.xlsx
tcdc.2014.4.9.xlsx
(63870, 7)
31
tcdc.2014.5.1.xlsx
tcdc.2014.5.10.xlsx
tcdc.2014.5.11.xlsx
tcdc.2014.5.12.xlsx
tcdc.2014.5.13.xlsx
tcdc.2014.5.14.xlsx
tcdc.2014.5.15.xlsx
tcdc.2014.5.16.xlsx
tcdc.2014.5.17.xlsx
tcdc.2014.5.18.xlsx
tcdc.2014.5.19.xlsx
tcdc.2014.5.2.xlsx
tcdc.2014.5.20.xlsx
tcdc.2014.5.21.xlsx
tcdc.2014.5.22.xlsx
tcdc.2014.5.23.xlsx
tcdc.2014.5.24.xlsx
tcdc.2014.5.25.xlsx
tcdc.2014.5.26.xlsx
tcdc.2014.5.27.xlsx
tcdc.2014.5.28.xlsx
tcdc.2014.5.29.xlsx
tcdc.2014.5.3.xlsx
tcdc.2014.5.30.xlsx
tcdc.2014.5.31.xlsx
tcdc.2014.5.4.xlsx
tcdc.2014.5.5.xlsx
tcdc.2014.5.6.xlsx
tcdc.2014.5.7.xlsx
tcdc.2014.5.8.xlsx
tcdc.2014.5.9.xlsx
(65999, 7)
30
tcdc.2014.6.1.xlsx
tcdc.2014.6.10.xlsx
tcdc.2014.6.11.xlsx
tcdc.2014.6.12.xlsx
tcdc.2014.6.13.xlsx
tcdc.2014.6.14.xlsx
tcdc.2014

tcdc.2015.5.4.xlsx
tcdc.2015.5.5.xlsx
tcdc.2015.5.6.xlsx
tcdc.2015.5.7.xlsx
tcdc.2015.5.8.xlsx
tcdc.2015.5.9.xlsx
(65999, 7)
30
tcdc.2015.6.1.xlsx
tcdc.2015.6.10.xlsx
tcdc.2015.6.11.xlsx
tcdc.2015.6.12.xlsx
tcdc.2015.6.13.xlsx
tcdc.2015.6.14.xlsx
tcdc.2015.6.15.xlsx
tcdc.2015.6.16.xlsx
tcdc.2015.6.17.xlsx
tcdc.2015.6.18.xlsx
tcdc.2015.6.19.xlsx
tcdc.2015.6.2.xlsx
tcdc.2015.6.20.xlsx
tcdc.2015.6.21.xlsx
tcdc.2015.6.22.xlsx
tcdc.2015.6.23.xlsx
tcdc.2015.6.24.xlsx
tcdc.2015.6.25.xlsx
tcdc.2015.6.26.xlsx
tcdc.2015.6.27.xlsx
tcdc.2015.6.28.xlsx
tcdc.2015.6.29.xlsx
tcdc.2015.6.3.xlsx
tcdc.2015.6.30.xlsx
tcdc.2015.6.4.xlsx
tcdc.2015.6.5.xlsx
tcdc.2015.6.6.xlsx
tcdc.2015.6.7.xlsx
tcdc.2015.6.8.xlsx
tcdc.2015.6.9.xlsx
(63870, 7)
31
tcdc.2015.7.1.xlsx
tcdc.2015.7.10.xlsx
tcdc.2015.7.11.xlsx
tcdc.2015.7.12.xlsx
tcdc.2015.7.13.xlsx
tcdc.2015.7.14.xlsx
tcdc.2015.7.15.xlsx
tcdc.2015.7.16.xlsx
tcdc.2015.7.17.xlsx
tcdc.2015.7.18.xlsx
tcdc.2015.7.19.xlsx
tcdc.2015.7.2.xlsx
tcdc.2015.7.20.xlsx
tcdc.2015

...

vwnd.10m.2011.12.13.xlsx
vwnd.10m.2011.12.14.xlsx
vwnd.10m.2011.12.15.xlsx
vwnd.10m.2011.12.16.xlsx
vwnd.10m.2011.12.17.xlsx
vwnd.10m.2011.12.18.xlsx
vwnd.10m.2011.12.19.xlsx
vwnd.10m.2011.12.2.xlsx
vwnd.10m.2011.12.20.xlsx
vwnd.10m.2011.12.21.xlsx
vwnd.10m.2011.12.22.xlsx
vwnd.10m.2011.12.23.xlsx
vwnd.10m.2011.12.24.xlsx
vwnd.10m.2011.12.25.xlsx
vwnd.10m.2011.12.26.xlsx
vwnd.10m.2011.12.27.xlsx
vwnd.10m.2011.12.28.xlsx
vwnd.10m.2011.12.29.xlsx
vwnd.10m.2011.12.3.xlsx
vwnd.10m.2011.12.30.xlsx
vwnd.10m.2011.12.31.xlsx
vwnd.10m.2011.12.4.xlsx
vwnd.10m.2011.12.5.xlsx
vwnd.10m.2011.12.6.xlsx
vwnd.10m.2011.12.7.xlsx
vwnd.10m.2011.12.8.xlsx
vwnd.10m.2011.12.9.xlsx
(65999, 7)
31
vwnd.10m.2012.1.1.xlsx
vwnd.10m.2012.1.10.xlsx
vwnd.10m.2012.1.11.xlsx
vwnd.10m.2012.1.12.xlsx
vwnd.10m.2012.1.13.xlsx
vwnd.10m.2012.1.14.xlsx
vwnd.10m.2012.1.15.xlsx
vwnd.10m.2012.1.16.xlsx
vwnd.10m.2012.1.17.xlsx
vwnd.10m.2012.1.18.xlsx
vwnd.10m.2012.1.19.xlsx
vwnd.10m.2012.1.2.xlsx
vwnd.10m.2012.1.20.xlsx
vwnd.10m.

vwnd.10m.2012.11.14.xlsx
vwnd.10m.2012.11.15.xlsx
vwnd.10m.2012.11.16.xlsx
vwnd.10m.2012.11.17.xlsx
vwnd.10m.2012.11.18.xlsx
vwnd.10m.2012.11.19.xlsx
vwnd.10m.2012.11.2.xlsx
vwnd.10m.2012.11.20.xlsx
vwnd.10m.2012.11.21.xlsx
vwnd.10m.2012.11.22.xlsx
vwnd.10m.2012.11.23.xlsx
vwnd.10m.2012.11.24.xlsx
vwnd.10m.2012.11.25.xlsx
vwnd.10m.2012.11.26.xlsx
vwnd.10m.2012.11.27.xlsx
vwnd.10m.2012.11.28.xlsx
vwnd.10m.2012.11.29.xlsx
vwnd.10m.2012.11.3.xlsx
vwnd.10m.2012.11.30.xlsx
vwnd.10m.2012.11.4.xlsx
vwnd.10m.2012.11.5.xlsx
vwnd.10m.2012.11.6.xlsx
vwnd.10m.2012.11.7.xlsx
vwnd.10m.2012.11.8.xlsx
vwnd.10m.2012.11.9.xlsx
(63870, 7)
31
vwnd.10m.2012.12.1.xlsx
vwnd.10m.2012.12.10.xlsx
vwnd.10m.2012.12.11.xlsx
vwnd.10m.2012.12.12.xlsx
vwnd.10m.2012.12.13.xlsx
vwnd.10m.2012.12.14.xlsx
vwnd.10m.2012.12.15.xlsx
vwnd.10m.2012.12.16.xlsx
vwnd.10m.2012.12.17.xlsx
vwnd.10m.2012.12.18.xlsx
vwnd.10m.2012.12.19.xlsx
vwnd.10m.2012.12.2.xlsx
vwnd.10m.2012.12.20.xlsx
vwnd.10m.2012.12.21.xlsx
vwnd.10m.2012.12.22.x

vwnd.10m.2013.10.17.xlsx
vwnd.10m.2013.10.18.xlsx
vwnd.10m.2013.10.19.xlsx
vwnd.10m.2013.10.2.xlsx
vwnd.10m.2013.10.20.xlsx
vwnd.10m.2013.10.21.xlsx
vwnd.10m.2013.10.22.xlsx
vwnd.10m.2013.10.23.xlsx
vwnd.10m.2013.10.24.xlsx
vwnd.10m.2013.10.25.xlsx
vwnd.10m.2013.10.26.xlsx
vwnd.10m.2013.10.27.xlsx
vwnd.10m.2013.10.28.xlsx
vwnd.10m.2013.10.29.xlsx
vwnd.10m.2013.10.3.xlsx
vwnd.10m.2013.10.30.xlsx
vwnd.10m.2013.10.31.xlsx
vwnd.10m.2013.10.4.xlsx
vwnd.10m.2013.10.5.xlsx
vwnd.10m.2013.10.6.xlsx
vwnd.10m.2013.10.7.xlsx
vwnd.10m.2013.10.8.xlsx
vwnd.10m.2013.10.9.xlsx
(65999, 7)
30
vwnd.10m.2013.11.1.xlsx
vwnd.10m.2013.11.10.xlsx
vwnd.10m.2013.11.11.xlsx
vwnd.10m.2013.11.12.xlsx
vwnd.10m.2013.11.13.xlsx
vwnd.10m.2013.11.14.xlsx
vwnd.10m.2013.11.15.xlsx
vwnd.10m.2013.11.16.xlsx
vwnd.10m.2013.11.17.xlsx
vwnd.10m.2013.11.18.xlsx
vwnd.10m.2013.11.19.xlsx
vwnd.10m.2013.11.2.xlsx
vwnd.10m.2013.11.20.xlsx
vwnd.10m.2013.11.21.xlsx
vwnd.10m.2013.11.22.xlsx
vwnd.10m.2013.11.23.xlsx
vwnd.10m.2013.11.24.x

vwnd.10m.2014.9.18.xlsx
vwnd.10m.2014.9.19.xlsx
vwnd.10m.2014.9.2.xlsx
vwnd.10m.2014.9.20.xlsx
vwnd.10m.2014.9.21.xlsx
vwnd.10m.2014.9.22.xlsx
vwnd.10m.2014.9.23.xlsx
vwnd.10m.2014.9.24.xlsx
vwnd.10m.2014.9.25.xlsx
vwnd.10m.2014.9.26.xlsx
vwnd.10m.2014.9.27.xlsx
vwnd.10m.2014.9.28.xlsx
vwnd.10m.2014.9.29.xlsx
vwnd.10m.2014.9.3.xlsx
vwnd.10m.2014.9.30.xlsx
vwnd.10m.2014.9.4.xlsx
vwnd.10m.2014.9.5.xlsx
vwnd.10m.2014.9.6.xlsx
vwnd.10m.2014.9.7.xlsx
vwnd.10m.2014.9.8.xlsx
vwnd.10m.2014.9.9.xlsx
(63870, 7)
31
vwnd.10m.2014.10.1.xlsx
vwnd.10m.2014.10.10.xlsx
vwnd.10m.2014.10.11.xlsx
vwnd.10m.2014.10.12.xlsx
vwnd.10m.2014.10.13.xlsx
vwnd.10m.2014.10.14.xlsx
vwnd.10m.2014.10.15.xlsx
vwnd.10m.2014.10.16.xlsx
vwnd.10m.2014.10.17.xlsx
vwnd.10m.2014.10.18.xlsx
vwnd.10m.2014.10.19.xlsx
vwnd.10m.2014.10.2.xlsx
vwnd.10m.2014.10.20.xlsx
vwnd.10m.2014.10.21.xlsx
vwnd.10m.2014.10.22.xlsx
vwnd.10m.2014.10.23.xlsx
vwnd.10m.2014.10.24.xlsx
vwnd.10m.2014.10.25.xlsx
vwnd.10m.2014.10.26.xlsx
vwnd.10m.2014.10.

vwnd.10m.2015.8.2.xlsx
vwnd.10m.2015.8.20.xlsx
vwnd.10m.2015.8.21.xlsx
vwnd.10m.2015.8.22.xlsx
vwnd.10m.2015.8.23.xlsx
vwnd.10m.2015.8.24.xlsx
vwnd.10m.2015.8.25.xlsx
vwnd.10m.2015.8.26.xlsx
vwnd.10m.2015.8.27.xlsx
vwnd.10m.2015.8.28.xlsx
vwnd.10m.2015.8.29.xlsx
vwnd.10m.2015.8.3.xlsx
vwnd.10m.2015.8.30.xlsx
vwnd.10m.2015.8.31.xlsx
vwnd.10m.2015.8.4.xlsx
vwnd.10m.2015.8.5.xlsx
vwnd.10m.2015.8.6.xlsx
vwnd.10m.2015.8.7.xlsx
vwnd.10m.2015.8.8.xlsx
vwnd.10m.2015.8.9.xlsx
(65999, 7)
30
vwnd.10m.2015.9.1.xlsx
vwnd.10m.2015.9.10.xlsx
vwnd.10m.2015.9.11.xlsx
vwnd.10m.2015.9.12.xlsx
vwnd.10m.2015.9.13.xlsx
vwnd.10m.2015.9.14.xlsx
vwnd.10m.2015.9.15.xlsx
vwnd.10m.2015.9.16.xlsx
vwnd.10m.2015.9.17.xlsx
vwnd.10m.2015.9.18.xlsx
vwnd.10m.2015.9.19.xlsx
vwnd.10m.2015.9.2.xlsx
vwnd.10m.2015.9.20.xlsx
vwnd.10m.2015.9.21.xlsx
vwnd.10m.2015.9.22.xlsx
vwnd.10m.2015.9.23.xlsx
vwnd.10m.2015.9.24.xlsx
vwnd.10m.2015.9.25.xlsx
vwnd.10m.2015.9.26.xlsx
vwnd.10m.2015.9.27.xlsx
vwnd.10m.2015.9.28.xlsx
vwnd.10m.201